In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：AutoML视频分类模型的批量预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_video_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_video_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来创建视频分类模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)进行批量预测。

数据集

本教程使用的数据集是来自麻省理工学院的[人体动作数据集](https://TODO)。您在本教程中将使用的数据集版本存储在一个公共云存储桶中。

### 目标

在本教程中，您将从一个Python脚本中创建一个AutoML视频分类模型，然后使用Vertex客户端库进行批量预测。您也可以使用`gcloud`命令行工具或在Google Cloud控制台上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

在使用批量预测和使用在线预测之间有一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行即时预测并实时返回结果。

* 批量预测服务：对背景中的整个实例集进行排队（批量）预测，并在准备就绪时将结果存储在Cloud Storage存储桶中。

### 成本

本教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

## 安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = '--user'
else:
    USER_FLAG = ''

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和谷歌*云存储*，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果您有这个选项，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得 $300 的免费信用额度用于计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud Notebook 中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 将以 `!` 为前缀的行作为 shell 命令运行，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  #@param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

地区

您还可以更改“REGION”变量，该变量用于笔记本的其余部分操作。以下是Vertex支持的地区。我们建议您选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶进行Vertex培训。并非所有地区都支持所有Vertex服务。有关每个地区的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = 'us-central1'  #@param {type: "string"}

时间戳

如果您在进行实时教程会话，您可能正在使用共享的测试账户或项目。为了避免用户之间在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并将其附加到将在此教程中创建的资源的名称中。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的Google Cloud帐户

**如果您正在使用Google Cloud Notebook**，您的环境已经通过身份验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证，通过oAuth进行账户认证。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐户访问项目的权限**部分，点击Role下拉列表。在过滤框中输入"Vertex"，然后选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，然后选择**Storage Object Admin**。

点击创建。一个包含您的密钥的JSON文件将下载到本地环境。

在下面的单元格中将路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

**无论您的笔记本环境如何，下面的步骤都是必需的。**

本教程旨在使用位于公共云存储桶中的训练数据以及本地云存储桶用于批量预测。您也可以使用存储在本地云存储桶中的自己的训练数据。

请在下面设置您的云存储桶名称。存储桶名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  #@param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在的情况下才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查Cloud Storage存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库和定义常量

导入 Vertex 客户端库

将 Vertex 客户端库导入到我们的 Python 环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的顶点 API 服务端点。
- `PARENT`：用于数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

自动机器学习常量

设置专属于自动机器学习数据集和训练的常量：

- 数据集模式：告知`数据集`资源服务数据集的类型。
- 数据标记（注释）模式：告知`数据集`资源服务数据如何标记（注释）。
- 数据集训练模式：告知`管道`资源服务要为哪种任务（例如，分类）训练模型。

In [ ]:
# Video Dataset type
DATA_SCHEMA = 'gs://google-cloud-aiplatform/schema/dataset/metadata/video_1.0.0.yaml'
# Video Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/video_classification_io_format_1.0.0.yaml"
# Video Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_video_classification_1.0.0.yaml"

#### 硬件加速器

设置硬件加速器（例如 GPU）用于预测。

设置变量 `DEPLOY_GPU/DEPLOY_NGPU` 以使用支持 GPU 的容器镜像，以及分配给虚拟机实例的 GPU 数量。例如，要使用一个 GPU 容器镜像，其中每个 VM 分配了 4 个 Nvidia Telsa K80 GPU，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定 `(None, None)` 来使用一个在 CPU 上运行的容器镜像。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, int(os.getenv("IS_TESTING_DEPOLY_GPU")))
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

容器（Docker）镜像

对于AutoML批量预测，端节点预测服务预先确定了用于服务二进制文件的容器镜像。更具体地说，该服务将根据您选择的硬件加速器为模型选择适当的容器。

#### 机器类型

接下来，设置用于预测的机器类型。

- 设置变量`DEPLOY_COMPUTE`来配置用于预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB内存。
     - `n1-highmem`: 每个vCPU 6.5GB内存
     - `n1-highcpu`: 每个vCPU 0.9GB内存
 - `vCPUs`: 数量 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = 'n1-standard'

VCPU = '4'
DEPLOY_COMPUTE = MACHINE_TYPE + '-' + VCPU
print('Deploy machine type', DEPLOY_COMPUTE)

# 教程

现在您已经准备好开始创建您自己的AutoML视频分类模型。

## 设置客户端

Vertex 客户端库作为客户端/服务器模型运行。在您的一端（Python 脚本）中，您将创建一个客户端，发送请求并接收来自 Vertex 服务器的响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置好它们。

- 数据集服务用于 `数据集` 资源。
- 模型服务用于 `模型` 资源。
- 流水线服务用于训练。
- 作业服务用于批量预测和自定义训练。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(
        client_options=client_options
    )
    return client


def create_model_client():
    client = aip.ModelServiceClient(
        client_options=client_options
    )
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(
        client_options=client_options
    )
    return client


def create_job_client():
    client = aip.JobServiceClient(
        client_options=client_options
    )
    return client


clients = {}
clients['dataset'] = create_dataset_client()
clients['model'] = create_model_client()
clients['pipeline'] = create_pipeline_client()
clients['job'] = create_job_client()

for client in clients.items():
    print(client)

数据集

现在您的客户已准备就绪，在训练模型的第一步是创建一个托管的数据集实例，然后将带标签的数据上传到其中。

### 创建`Dataset`资源实例

使用辅助函数`create_dataset`来创建`Dataset`资源的实例。此函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个Vertex `Dataset`资源（`aip.Dataset`），具有以下参数：
 - `display_name`：您选择的可读名称。
 - `metadata_schema_uri`：数据集类型的模式。
3. 调用客户端数据集服务方法`create_dataset`，具有以下参数：
 - `parent`：您的`Database`、`Model`和`Endpoint`资源的Vertex位置根路径。
 - `dataset`：您创建的Vertex数据集对象实例。
4. 该方法返回一个`operation`对象。

`operation`对象是Vertex处理长时间运行操作的异步调用方式。虽然此步骤通常很快，但当您在项目中首次使用它时，由于供应问题可能会导致较长延迟。

您可以使用`operation`对象来获取操作的状态（例如，创建`Dataset`资源）或取消操作，通过调用操作方法：

| 方法         | 描述             |
| ----------- | ----------- |
| result()    | 等待操作完成并以JSON格式返回结果对象。      |
| running()   | 返回操作是否仍在运行的True/False值。        |
| done()      | 返回操作是否已完成的True/False值。 |
| canceled()  | 返回操作是否已取消的True/False值。 |
| cancel()    | 取消操作（可能需要最多30秒）。 |

In [ ]:
TIMEOUT = 90

def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(display_name=name, metadata_schema_uri=schema, labels=labels)

        operation = clients['dataset'].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("hmdb,tst-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split('/')[-1]

print(dataset_id)

### 数据准备

视频的Vertex `Dataset`资源对您的数据有一些要求。

- 视频必须存储在云存储桶中。
- 每个视频文件必须是视频格式（MPG，AVI，...）。
- 必须在您的云存储桶中存储一个包含每个视频路径和标签的索引文件。
- 索引文件必须是CSV或JSONL格式。

#### CSV

对于视频分类，CSV索引文件有一些要求：

- 没有标题。
- 第一列是视频的云存储路径。
- 第二列是标签。

#### 云存储培训数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = 'gs://automl-video-demo-data/hmdb_split1_5classes_train_inf.csv'

快速查看您的数据

您将使用存储在公共云存储桶中的MIT Human Motion数据集的一个版本，使用CSV索引文件。

首先快速查看数据。通过在CSV索引文件中计算行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if 'IMPORT_FILES' in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的 Vertex 数据集资源中。使用这个辅助函数 `import_data` 来导入数据。该函数执行以下操作：

- 使用 `Dataset` 客户端。
- 调用客户端方法 `import_data`，并提供以下参数：
  - `name`：您为 `Dataset` 资源命名的可读名称（例如，hmdb，tst）。
  - `import_configs`：导入配置。

- `import_configs`：一个包含字典的 Python 列表，其中包含键/值条目：
  - `gcs_sources`：一个包含一个或多个索引文件路径的 URI 列表。
  - `import_schema_uri`：标识标签类型的模式。

`import_data()` 方法返回一个长时间运行的 `operation` 对象。这将需要几分钟来完成。如果您在进行实时教程，这是一个很好的时间提问或休息一下。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{
        'gcs_source': {'uris': gcs_sources},
        'import_schema_uri': schema
    }]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients['dataset'].import_data(name=dataset_id, import_configs=config)
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print("after: running:", operation.running(), "done:", operation.done(), "cancelled:", operation.cancelled())

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

训练模型

现在使用您的Vertex“数据集”资源来训练一个AutoML视频分类模型。要训练模型，请执行以下步骤：

1. 为“数据集”资源创建一个Vertex训练管道。
2. 执行管道来开始训练。

### 创建一个训练流水线

你可能会问，我们用流水线做什么？当工作（比如训练）包含多个步骤时，通常按顺序进行：执行步骤A，执行步骤B，等等。通过将这些步骤放入流水线中，我们可以得到以下好处：

1. 可以重复使用于后续训练工作。
2. 可以被打包并作为批处理任务运行。
3. 可以进行分布式处理。
4. 所有步骤都和同一个流水线任务相关联，以便跟踪进度。

使用这个辅助函数 `create_pipeline`，它接受以下参数：

- `pipeline_name`：给流水线任务的人类可读名称。
- `model_name`：模型的人类可读名称。
- `dataset`：Vertex 完全限定的数据集标识符。
- `schema`：数据集标注（注释）训练模式。
- `task`：描述训练工作要求的字典。

这个辅助函数调用 `Pipeline` 客户端服务的 `create_pipeline` 方法，该方法接受以下参数：

- `parent`：`Dataset`，`Model`，和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：流水线训练工作的完整规格。

现在让我们深入了解创建 `training_pipeline` 规格的*最小*要求：

- `display_name`：给流水线任务的人类可读名称。
- `training_task_definition`：数据集标注（注释）训练模式。
- `training_task_inputs`：描述训练工作要求的字典。
- `model_to_upload`：模型的人类可读名称。
- `input_data_config`：数据集规格。
 - `dataset_id`：仅限Vertex数据集标识符（非完全限定）--这是完全限定标识符的最后一部分。
 - `fraction_split`：如果指定了，用于训练、测试和验证的数据集百分比。否则，百分比将由AutoML自动选择。
  - 注意，对于视频来说，不支持验证集分割--只支持训练和测试。

In [ ]:
 def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split('/')[-1]

    input_config = {'dataset_id': dataset_id,
                    'fraction_split': {
                        'training_fraction': 0.8,
                        'test_fraction': 0.2
                    }}

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients['pipeline'].create_training_pipeline(parent=PARENT, training_pipeline=training_pipeline)
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务需求

接下来，构建任务需求。与其他参数不同，需要一个 Google protobuf 结构体作为`task`字段的参数，而不是 Python（类似 JSON 格式）字典。使用`json_format.ParseDict`方法进行转换。

对于视频分类，没有必需的最小字段需要指定。

最后，通过调用辅助函数`create_pipeline`来创建管道，该函数会返回一个训练管道对象的实例。

In [ ]:
PIPE_NAME = "hmdb,tst_pipe-" + TIMESTAMP
MODEL_NAME = "hmdb,tst_model-" + TIMESTAMP

task = json_format.ParseDict({}, Value())

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练管道的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split('/')[-1]

print(pipeline_id)

### 获取关于训练管道的信息

现在只获取此训练管道实例的管道信息。通过调用作业客户服务的`get_training_pipeline`方法，使用以下参数helper函数获取该作业的信息：

- `name`：Vertex完全合格的管道标识符。

当模型训练完成时，管道状态将为`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients['pipeline'].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

部署

训练上述模型可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过将“结束时间”减去“开始时间”来计算训练模型所花费的实际时间。对于您的模型，您需要知道完全限定的Vertex模型资源标识符，这是管道服务分配给它的。您可以从返回的管道实例中获取此信息，作为字段“model_to_deploy.name”。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

模型信息

现在您的模型已经训练完成，您可以获取一些关于您的模型的信息。

评估模型资源

现在找出模型服务认为您的模型有多好。作为训练的一部分，数据集的一部分被设置为测试（保留）数据，Pipeline 服务使用这部分数据来评估模型。

### 列出所有切片的评估

使用这个辅助函数 `list_model_evaluations`，它接受以下参数：

- `name`：`Model` 资源的 Vertex 完全限定模型标识符。

这个辅助函数使用模型客户端服务的 `list_model_evaluations` 方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估（您可能只有一个），我们将打印出每个评估中每个指标的所有键名称，并且对于一个小的集合（`auPrc`），您将打印出结果。

In [ ]:
def list_model_evaluations(name):
    response = clients['model'].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print('auPrc', metrics['auPrc'])


    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

## 批量预测模型部署

现在部署您为批量预测创建的已训练的Vertex `Model`资源。这与部署用于按需预测的`Model`资源不同。

对于在线预测，您需要：

1. 创建一个用于部署`Model`资源的`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

3. 对`Endpoint`资源发出在线预测请求。

对于批处理预测，您需要：

1. 创建一个批处理预测作业。

2. 作业服务将为批处理预测请求分配资源。

3. 返回批处理预测请求的结果给调用者。

4. 作业服务将取消为批处理预测请求分配的资源。

做一个批量预测请求，现在对您部署的模型进行批量预测。

### 获取测试项目

现在对您的Vertex模型进行批量预测。您将使用数据集中的任意示例作为测试项目。不必担心这些示例可能已用于训练模型，我们只是想展示如何进行预测。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2

if len(test_items[0]) == 5:
    _, test_item_1, test_label_1, _, _ = str(test_items[0]).split(',')
    _, test_item_2, test_label_2, _, _ = str(test_items[1]).split(',')
else:
    test_item_1, test_label_1, _, _ = str(test_items[0]).split(',')
    test_item_2, test_label_2, _, _ = str(test_items[1]).split(',')


print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

制作批量输入文件

现在制作一个批量输入文件，将其存储在您的本地云存储桶中。批量输入文件可以是CSV也可以是JSONL。您将在本教程中使用JSONL。对于JSONL文件，您为每个视频的每一行创建一个字典条目。该字典包含键/值对：

- `content`：视频的云存储路径。
- `mimeType`：内容类型。在我们的示例中，它是一个`avi`文件。
- `timeSegmentStart`：要进行预测的视频中的起始时间戳。*注意*，时间戳必须以字符串形式指定，并后跟s（秒）、m（分钟）或h（小时）。
- `timeSegmentEnd`：视频中要进行预测的结束时间戳。

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_NAME + '/test.jsonl'
with tf.io.gfile.GFile(gcs_input_uri, 'w') as f:
    data = { "content": test_item_1, "mimeType": "video/avi", "timeSegmentStart": "0.0s", 'timeSegmentEnd': '5.0s' }
    f.write(json.dumps(data) + '\n')
    data = { "content": test_item_2, "mimeType": "video/avi", "timeSegmentStart": "0.0s", 'timeSegmentEnd': '5.0s' }
    f.write(json.dumps(data) + '\n')

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 计算实例扩展

处理批量预测请求时，您有几种计算实例扩展的选择：

- 单个实例：批量预测请求在单个计算实例上处理。
  - 将最小（`MIN_NODES`）和最大（`MAX_NODES`）计算实例数设置为一。

- 手动扩展：将批量预测请求分配到您手动指定的固定数量的计算实例上。
  - 将最小（`MIN_NODES`）和最大（`MAX_NODES`）计算实例数设置为相同数量的节点。当模型首次部署到实例时，将提供固定数量的计算实例，并且批量预测请求会均匀分配到这些实例上。

- 自动扩展：将批量预测请求分配到可扩展数量的计算实例上。
  - 设置最小（`MIN_NODES`）计算实例数，以便在模型首次部署时提供和取消提供，并根据负载情况设置最大（`MAX_NODES`）计算实例数。

最小计算实例数对应于`min_replica_count`字段，最大计算实例数对应于`max_replica_count`字段，在您后续的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 创建批量预测请求

现在您的两个测试项目的批次已经准备好了，让我们进行批量请求。使用以下参数调用这个辅助函数 `create_batch_prediction_job`：

- `display_name`：预测作业的人类可读名称。
- `model_name`：`Model` 资源的 Vertex 完全限定标识符。
- `gcs_source_uri`：输入文件的 Cloud Storage 路径，即您之前创建的文件。
- `gcs_destination_output_uri_prefix`：服务将写入预测结果的 Cloud Storage 路径。
- `parameters`：用于服务预测结果的额外过滤参数。

辅助函数调用作业客户端服务的 `create_batch_prediction_job` 方法，使用以下参数：

- `parent`：Dataset、Model 和 Pipeline 资源的 Vertex 位置根路径。
- `batch_prediction_job`：批量预测作业的规格。

现在让我们深入了解 `batch_prediction_job` 的规格：

- `display_name`：用于预测批量作业的人类可读名称。
- `model`：`Model` 资源的 Vertex 完全限定标识符。
- `dedicated_resources`：为批量预测作业提供的计算资源。
  - `machine_spec`：要提供的计算实例。使用您之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则仅分配 CPU。
  - `starting_replica_count`：最初要提供的计算实例数，即您之前设置的变量 `MIN_NODES`。
  - `max_replica_count`：要扩展到的最大计算实例数，即您之前设置的变量 `MAX_NODES`。
- `model_parameters`：用于服务预测结果的额外过滤参数。
 - `confidenceThreshold`：进行预测时的最小置信度阈值。
 - `maxPredictions`：按置信度排序返回每个分类的最大预测数。
 - `oneSecIntervalClassification`：如果为 `True`，则在一秒间隔上进行预测。
 - `shotClassification`：如果为 `True`，则在每个摄像头镜头边界上进行预测。
 - `segmentClassification`：如果为 `True`，则在每个时间段上进行预测；否则将为整个时间段进行预测。
- `input_config`：要预测的实例的输入源和格式类型。
 - `instances_format`：批量预测请求文件的格式：`csv` 或 `jsonl`。
 - `gcs_source`：一个或多个云存储路径列表，指向您的批量预测请求。
- `output_config`：预测结果的输出目标和格式。
 - `prediction_format`：批量预测响应文件的格式：仅为 `jsonl`。
 - `gcs_destination`：预测结果的输出目标。

您可能会问，置信度阈值如何影响模型准确性？这个阈值不会改变准确性。它改变的是召回率和精确率。

    - 精确率：精确率越高，预测正确的可能性越大，但返回的预测数较少。增加置信度阈值可提高精确率。
    - 召回率：召回率越高，结果中正确预测的可能性越大，但会返回更多带有错误预测的预测。减小置信度阈值可提高召回率。

在这个例子中，您将预测精确率。您将置信度阈值设为 0.5，对一个操作的最大预测数也设为两个。由于各类别的置信度值必须相加为一，只有两种可能的结果：

    1. 出现平局，都是 0.5，并返回两个预测。
    2. 一个值高于 0.5，其余值低于 0.5，并返回一个预测。

这个调用是一个异步操作。您将从响应对象中打印一些选择字段，包括：

- `name`：分配给批量预测作业的 Vertex 完全限定标识符。
- `display_name`：预测批量作业的人类可读名称。
- `model`：Model 资源的 Vertex 完全限定标识符。
- `generate_explanations`：是否提供了预测的解释（可解释性）。
- `state`：预测作业的状态（挂起、运行等）。

由于这个调用需要一些时间来执行，您可能会看到 `state` 为 `JobState.JOB_STATE_PENDING`。

In [ ]:
BATCH_MODEL = "hmdb,tst_batch-" + TIMESTAMP


def create_batch_prediction_job(display_name, model_name, gcs_source_uri, gcs_destination_output_uri_prefix, parameters=None):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    batch_prediction_job = {
        "display_name": display_name,
        # Format: 'projects/{project}/locations/{location}/models/{model_id}'
        "model": model_name,
        "model_parameters": json_format.ParseDict(parameters, Value()),
        "input_config": {
            "instances_format": IN_FORMAT,
            "gcs_source": {"uris": [gcs_source_uri]},
        },
        "output_config": {
            "predictions_format": OUT_FORMAT,
            "gcs_destination": {"output_uri_prefix": gcs_destination_output_uri_prefix},
        },
        "dedicated_resources": {
            "machine_spec": machine_spec,
            "starting_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES
        }

    }
    response = clients['job'].create_batch_prediction_job(
        parent=PARENT, batch_prediction_job=batch_prediction_job
    )
    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", response.labels)
    return response


IN_FORMAT = 'jsonl'
OUT_FORMAT = 'jsonl'  # [jsonl]

response = create_batch_prediction_job(BATCH_MODEL, model_to_deploy_id, gcs_input_uri, BUCKET_NAME, None)

现在获取您创建的批量预测作业的唯一标识符。

In [ ]:
# The full unique ID for the batch job
batch_job_id = response.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split('/')[-1]

print(batch_job_id)

### 获取有关批量预测作业的信息

使用此辅助函数`get_batch_prediction_job`，并提供以下参数：

- `job_name`：批量预测作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的`get_batch_prediction_job`方法，并提供以下参数：

- `name`：批量预测作业的 Vertex 完全限定标识符。在本教程中，您将传递给它您的批量预测作业的 Vertex 完全限定标识符 -- `batch_job_id`

辅助函数将返回存储预测结果的云存储路径 -- `gcs_destination`。

In [ ]:
def get_batch_prediction_job(job_name, silent=False):
    response = clients['job'].get_batch_prediction_job(name=job_name)
    if silent:
        return response.output_config.gcs_destination.output_uri_prefix, response.state

    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:   # not all data types support explanations
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" error:", response.error)
    gcs_destination = response.output_config.gcs_destination
    print(" gcs_destination")
    print("  output_uri_prefix:", gcs_destination.output_uri_prefix)
    return gcs_destination.output_uri_prefix, response.state


predictions, state = get_batch_prediction_job(batch_job_id)

获取预测结果

当批量预测处理完成时，作业状态将为`JOB_STATE_SUCCEEDED`。

最后，您可以查看存储在Cloud Storage路径中的预测结果，这个输出路径是您在创建批量预测作业时设置的。预测结果将以JSONL格式呈现，这是您在创建批量预测作业时指定的格式，在一个名为`prediction`的子文件夹下，该文件夹下将有一个名为`predictions*.jsonl`的文件。

现在显示（cat）内容。您会看到多个JSON对象，每个对象对应一个预测。

对于每个预测：

- `content`：用于预测请求的视频。
- `displayName`：预测动作。
- `confidence`：预测的置信度，介于0和1之间。
- `timeSegmentStart`/`timeSegmentEnd`：预测动作的开始和结束时间偏移量。

In [ ]:
def get_latest_predictions(gcs_out_dir):
    ''' Get the latest prediction subfolder using the timestamp in the subfolder name'''
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split('/')[-2]
        if subfolder.startswith('prediction-'):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    predictions, state = get_batch_prediction_job(batch_job_id, True)
    if state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", state)
        if state == aip.JobState.JOB_STATE_FAILED:
            raise Exception("Batch Job Failed")
    else:
        folder = get_latest_predictions(predictions)
        ! gsutil ls $folder/prediction*.jsonl

        ! gsutil cat $folder/prediction*.jsonl
        break
    time.sleep(60)

清理

要清理本项目中使用的所有GCP资源，您可以删除用于教程的GCP项目。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and 'dataset_id' in globals():
        clients['dataset'].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and 'pipeline_id' in globals():
        clients['pipeline'].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and 'model_to_deploy_id' in globals():
        clients['model'].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and 'endpoint_id' in globals():
        clients['endpoint'].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and 'batch_job_id' in globals():
        clients['job'].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and 'job_id' in globals():
        clients['job'].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and 'hpt_job_id' in globals():
        clients['job'].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r $BUCKET_NAME